In [1]:
import cv2
import numpy as np

In [2]:
frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)

def empty(a):
    pass

cv2.namedWindow("Image Filter")
cv2.resizeWindow("Image Filter", 320, 380)
cv2.createTrackbar("PIXEL ART FILTER", "Image Filter",0,1,empty)
cv2.createTrackbar("DRAWING FILTER", "Image Filter",0,1,empty)
cv2.createTrackbar("OILPAINT FILTER", "Image Filter",0,1,empty)
cv2.createTrackbar("CARTOON FILTER", "Image Filter",0,1,empty)
cv2.createTrackbar("PENCIL SKETCH FILTER", "Image Filter",0,1,empty)
cv2.createTrackbar("COLOR SKETCH FILTER", "Image Filter",0,1,empty)
cv2.createTrackbar("EDGE PRESERVING FILTER", "Image Filter",0,1,empty)
cv2.createTrackbar("WATERCOLOR FILTER", "Image Filter",0,1,empty)
cv2.createTrackbar("PIXEL WATERCOLOR FILTER", "Image Filter",0,1,empty)
    
frame_counter = 0 
while True : 
    _, frame = cap.read()

    result = frame.copy()

    px = cv2.getTrackbarPos("PIXEL ART FILTER", "Image Filter")
    if px == 0:
        pass
    else:
        result = cv2.resize(frame,(150,120))
        result = cv2.resize(result,(frameWidth,frameHeight),interpolation=cv2.INTER_NEAREST)

    dw = cv2.getTrackbarPos("DRAWING FILTER", "Image Filter")
    if dw == 0:
        pass
    else:
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (21,21), 0)
        result = cv2.divide(gray, blur, scale = 256.0)

    op = cv2.getTrackbarPos("OILPAINT FILTER", "Image Filter")
    if op == 0:
        pass
    else:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        morph = cv2.morphologyEx(frame, cv2.MORPH_OPEN, kernel)
        result = cv2.normalize(morph,None,20,255,cv2.NORM_MINMAX)

    ct = cv2.getTrackbarPos("CARTOON FILTER", "Image Filter")
    if ct == 0:
        pass
    else:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        grayblur = cv2.medianBlur(gray, 5)
        edges = cv2.adaptiveThreshold(grayblur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
        color = cv2.bilateralFilter(frame, 9, 300, 300)
        result = cv2.bitwise_and(color, color, mask=edges)

    ps = cv2.getTrackbarPos("PENCIL SKETCH FILTER", "Image Filter")
    if ps == 0:
        pass
    else:
        ps_gray, ps_color = cv2.pencilSketch(frame, sigma_s=60, sigma_r=0.07, shade_factor=0.05) 
        result = ps_gray
    
    cs = cv2.getTrackbarPos("COLOR SKETCH FILTER", "Image Filter")
    if cs == 0:
        pass
    else:
        ps_gray, ps_color = cv2.pencilSketch(frame, sigma_s=60, sigma_r=0.07, shade_factor=0.05) 
        result = ps_color
    
    ep = cv2.getTrackbarPos("EDGE PRESERVING FILTER", "Image Filter")
    if ep == 0:
        pass
    else:
        result = cv2.edgePreservingFilter(frame, flags=1, sigma_s=60, sigma_r=0.4)

    wt = cv2.getTrackbarPos("WATERCOLOR FILTER", "Image Filter")
    if wt == 0:
        pass
    else:
        result = cv2.stylization(frame, sigma_s=10, sigma_r=0.5)

    pw = cv2.getTrackbarPos("PIXEL WATERCOLOR FILTER", "Image Filter")
    if pw == 0:
        pass
    else:
        spx = cv2.resize(result,(230,210))
        pxi = cv2.resize(spx,(frameWidth,frameHeight),interpolation=cv2.INTER_NEAREST)
        result = cv2.stylization(pxi, sigma_s=10, sigma_r=0.5)

    output = result.copy()

    hStack = np.hstack([output])
    cv2.imshow("Real-Time Art Filter System", hStack)

    if cv2.waitKey(1) & 0xFF == ord('s'): 
        frame_name = "art_filter_frame_{}.png".format(frame_counter)
        cv2.imwrite(frame_name, output,)
        print("{} written!".format(frame_name))
        frame_counter += 1

    elif cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()